### 0 - Load text file

In [2]:
path = "./pseudo_podcast.txt"
txt_list = []
with open(path) as f:
    line = f.readline()
    txt_list.append(line)
    while line:
        line = f.readline()
        txt_list.append(line)

NameError: name 'path' is not defined

### 1 - Exclude all stopwords from the text file

In [6]:
import nltk
from nltk.corpus import stopwords

print(stopwords.words('english'))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/michaelzhang/nltk_data'
    - '/Users/michaelzhang/miniconda3/nltk_data'
    - '/Users/michaelzhang/miniconda3/share/nltk_data'
    - '/Users/michaelzhang/miniconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


### 2 - Top 10 Frequent words

In [ ]:
freq = {}
freq[keyword] += 1

from heapq import heappop, heappush, heapify 
  
# Creating empty heap 
heap = [] 
heapify(heap) 

heappush(heap, KV)

heappop(heap)

### 3 - Calculate tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
'This is the first document.',
'This document is the second document.',
'And this is the third one.',
'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
# ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

### 4 - Load Mapping from a JSON

In [ ]:
import json

with open('path_to_file/person.json') as f:
  data = json.load(f)

### 5 - Vote by 10 top genre & Crop top 3 

In [ ]:
scores = {}
for genre in genres:
    scores[genre] += score
    